<a href="https://colab.research.google.com/github/GavinButts/Fall-Data-Challenge-2022/blob/main/Code/Kevin/Ordinal%20-%20SEGRADES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns
!pip install --upgrade --no-deps statsmodels
import scipy.stats as stats


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.8 MB 6.8 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstalled statsmodels-0.12.2


In [41]:
sheet_id = "11rno0SjsOyV1JlevH3q40UMm1s5e6m7g"
sheet_name = "curated_2019-required"

url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df = pd.read_csv(url)
print(df)

            BASMID  ALLGRADEX  EDCPUB  SCCHOICE  SPUBCHOIX  SCONSIDR  \
0      20191000012         12       1         2          3         2   
1      20191000029          4       1         2          3         2   
2      20191000059         11       2         2          2         1   
3      20191000070          6       1         1          1         1   
4      20191000078          9       1         2          3         2   
...            ...        ...     ...       ...        ...       ...   
15495  20191225472          7       1         2          2         2   
15496  20191225475          2       1         2          2         2   
15497  20191225477         12       1         1          3         2   
15498  20191225479         13       1         2          2         1   
15499  20191225500         14       1         1          2         2   

       SCHLHRSWK  EINTNET  MOSTIMPT  INTNUM  ...  SEFUTUREX  DSBLTY  \
0              4        4        -1      -1  ...          6     

In [42]:
#create FOYESCNT variable that determines the amount of times each person said 'Yes' on the FO#### questions (ITEMS 72-74)
#Legend:  Number of 'Yes'

FOYESCNT = []      #variable that will be added to csv
yesCount = 0
for i in range(15500):
  if df['FOSTORY2X'][i] == 1:
    yesCount += 1
  if df['FOCRAFTS'][i] == 1:
    yesCount += 1
  if df['FOGAMES'][i] == 1:
    yesCount += 1
  if df['FOBUILDX'][i] == 1:
    yesCount += 1
  if df['FOSPORT'][i] == 1:
    yesCount += 1
  if df['FORESPON'][i] == 1:
    yesCount += 1
  if df['FOHISTX'][i] == 1:
    yesCount += 1
  if df['FOLIBRAYX'][i] == 1:
    yesCount += 1
  if df['FOBOOKSTX'][i] == 1:
    yesCount += 1
  if df['FODINNERX'][i] != 0:
    yesCount += 1
  FOYESCNT.append(yesCount)
  yesCount = 0

df["FOYESCNT"] = FOYESCNT
df.to_csv("sample.csv", index=False)  #adds variable to csv

#create FSYESCNT variable that determines the amount of times each person said 'Yes' on the FS#### questions (ITEMS 60)
#Legend:  Number of 'Yes'

FSYESCNT = []      #variable that will be added to csv
yesCount = 0
FSYESCNT_LMH = []

for i in range(15500):
  if df['FSSPORTX'][i] == 1:
    yesCount += 1
  if df['FSVOL'][i] == 1:
    yesCount += 1
  if df['FSMTNG'][i] == 1:
    yesCount += 1
  if df['FSPTMTNG'][i] == 1:
    yesCount += 1
  if df['FSATCNFN'][i] == 1:
    yesCount += 1
  if df['FSFUNDRS'][i] == 1:
    yesCount += 1
  if df['FSCOMMTE'][i] == 1:
    yesCount += 1
  if df['FSCOUNSLR'][i] == 1:
    yesCount += 1
  FSYESCNT.append(yesCount)
  if yesCount <= 3:
    FSYESCNT_LMH.append('1) LOW INVOLVMENT')
  elif yesCount > 3 and yesCount<=5:
    FSYESCNT_LMH.append('2) MEDIUM INVOLVEMENT')
  elif yesCount > 5:
    FSYESCNT_LMH.append('3) HIGH INVOLVEMENT')
  yesCount = 0

df["FSYESCNT"] = FSYESCNT
df["FSYESCNT_LMH"] = FSYESCNT_LMH

df.to_csv("sample.csv", index=False)  #adds variable to csv


In [43]:
# delete all rows for students who answered 5 or -1
'''
df_clean = []
for i in range(len(df)):
  if !(df.SEGRADES[i] == -1 or df.SEGRADES[i] == 5):
    df_clean.append(df[]) = df.drop(i)
'''


i = 0
L = len(df.SEGRADES)
while i < L:
  if (df.SEGRADES[i] == -1) or (df.SEGRADES[i] == 5):
    df = df.drop(i)
  i+=1


In [44]:
print(df['SEGRADES'])


0        1
2        3
3        1
4        1
6        1
        ..
15494    4
15495    1
15497    1
15498    1
15499    2
Name: SEGRADES, Length: 13533, dtype: int64


In [45]:
#Data Pre-Processing
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=[1,2,3,4], ordered=True)
df["SEGRADES"] = df["SEGRADES"].astype(cat_type)
df["SEGRADES"].dtype

CategoricalDtype(categories=[1, 2, 3, 4], ordered=True)

In [46]:
from statsmodels.miscmodels.ordinal_model import OrderedModel

In [51]:
from os import XATTR_CREATE
#['HHPARN19X','FHPLACE','SEABSNT','FSYESCNT','SEGRADES','FOYESCNT','RACEETH','TTLHHINC']
x = ['ALLGRADEX','EDCPUB','SEGRADEQ','FOYESCNT',
     'SEABSNT','RACEETH','OWNRNTHB','SEFUTUREX']
mod_prob = OrderedModel(df['SEGRADES'],
                        df[x],
                        distr='logit')
 
res_log = mod_prob.fit(method='bfgs')
res_log.summary()


Optimization terminated successfully.
         Current function value: 0.640083
         Iterations: 43
         Function evaluations: 45
         Gradient evaluations: 45


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:               SEGRADES   Log-Likelihood:                -8662.2
Model:                   OrderedModel   AIC:                         1.735e+04
Method:            Maximum Likelihood   BIC:                         1.743e+04
Date:                Fri, 28 Oct 2022                                         
Time:                        22:00:36                                         
No. Observations:               13533                                         
Df Residuals:                   13522                                         
Df Model:                          11                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ALLGRADEX      0.0783      0.006     12.103      0.000       0.066       0.091
EDCPUB        -0.3225      0.071     -4.564      0.000      -0.461      -0.184
SEGRADEQ       2.2261      0.034     66.430      0.000       2.160       2.292
FOYESCNT       0.0324      0.010      3.279      0.001       0.013       0.052
SEABSNT        0.2942      0.035      8.358      0.000       0.225       0.363
RACEETH        0.0995      0.016      6.097      0.000       0.068       0.132
OWNRNTHB       0.3094      0.042      7.305      0.000       0.226       0.392
SEFUTUREX     -0.2469      0.018    -13.655      0.000      -0.282      -0.211
1/2            5.1476      0.195     26.410      0.000       4.766       5.530
2/3            1.1590      0.014     80.072      0.000       1.131       1.187
3/4            1.0459      0.027     38.395      0.000       0.993       1.099
==============================================================================
"""

In [52]:
predicted = res_log.model.predict(res_log.params, exog=df[x])
predicted


/usr/local/lib/python3.7/dist-packages/statsmodels/miscmodels/ordinal_model.py:419: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  xb = xb[:, None]


array([[9.07238077e-01, 8.85560626e-02, 3.96062043e-03, 2.45240182e-04],
       [1.72091022e-02, 2.80492866e-01, 5.81797695e-01, 1.20500337e-01],
       [9.53895285e-01, 4.41121379e-02, 1.87663478e-03, 1.15942847e-04],
       ...,
       [5.30318681e-01, 4.34387531e-01, 3.31735029e-02, 2.12028495e-03],
       [9.19521862e-01, 7.68757919e-02, 3.39241579e-03, 2.09929925e-04],
       [1.22667869e-01, 6.49270317e-01, 2.11192626e-01, 1.68691887e-02]])

In [53]:
pred_choice = predicted.argmax(1)
print('Fraction of correct choice predictions')
print((np.asarray(df['SEGRADES'].values.codes) == pred_choice).mean())


Fraction of correct choice predictions
0.717948717948718
